In [48]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Dense, Dropout, Activation, Reshape, Lambda, Merge, Embedding
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

In [4]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

if K.image_dim_ordering() == 'th':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

X_train shape: (60000, 1, 28, 28)
60000 train samples
10000 test samples


In [73]:
batch_size = 128
nb_classes = 10
nb_epoch = 2
# number of convolutional filters to use
nb_filters = 16
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (5, 5)

In [90]:
import theano.tensor as T
from theano.tensor import *
from theano import function
from theano import pp
x = T.dscalar('x')
y = T.dscalar('y')
z = x + y
f = function([x, y], z)
print(f(2, 3))
print(pp(z))
z.eval({x : 2, y : 3})

5.0
(x + y)


array(5.0)

In [145]:
np.random.seed(12)
a = np.random.choice(100, 24, replace=False)
a = a.reshape((6,4))
x = T.dmatrix('x')
args = T.argsort(x, axis=1)
k=2
kargs = args[:,-k:]
kargs = T.sort(kargs)
kargs = kargs.flatten()
# z = kargs.shape
maxf = function([x],kargs)
print(a.shape)
print(a)
print(maxf(a))


(6, 4)
[[17 41 92 14]
 [68 31 89 15]
 [21 60 12  8]
 [39  9  7 70]
 [58 24 86 16]
 [83 55 26 54]]
[1 2 0 2 0 1 0 3 0 2 0 1]


In [74]:
model = Sequential()
def kmax(x):
    index = T.argsort(x, axis=3)

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
kmax_layer = Lambda(function=kmax, output_shape=lambda shape: (shape[0], shape[1], shape[3]))
model.add(kmax_layer)
# model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

In [75]:
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 19s - loss: 0.5924 - acc: 0.8063 - val_loss: 0.1169 - val_acc: 0.9638
Epoch 2/2
60000/60000 [==============================] - 18s - loss: 0.2141 - acc: 0.9367 - val_loss: 0.0880 - val_acc: 0.9718
Test score: 0.0879899788989
Test accuracy: 0.9718


In [40]:
def get_activations(model, layer, X_batch):
    get_activations = K.function([model.layers[0].input, K.learning_phase()], [model.layers[layer].output])
    activations = get_activations([X_batch,0])
    return activations

In [60]:
X_train[0].shape
model.layers[3]

In [82]:
test=X_train[:3]
print(model.layers[0])
print(model.layers[3])
print(model.layers[0].input)
print(model.layers[0].input.type)
res=get_activations(model, 3, test)[0]
print(test.shape)
print(res.shape)
type(res[0][0])
print(res[0][0][4])
res2=get_activations(model, 4, test)[0]
print(res2[0][0][4])

convolution2d_input_10
TensorType(float32, 4D)
(3, 1, 28, 28)
(3, 16, 20, 20)
[ 0.          0.          0.          0.          0.06168696  0.19090232
  0.          0.          0.          0.          0.          0.12789133
  0.01868857  0.          0.          0.04749847  0.18658826  0.09290097
  0.00366398  0.        ]
0.190902


In [52]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_7 (Convolution2D)  (None, 16, 24, 24)    416         convolution2d_input_4[0][0]      
____________________________________________________________________________________________________
activation_11 (Activation)       (None, 16, 24, 24)    0           convolution2d_7[0][0]            
____________________________________________________________________________________________________
convolution2d_8 (Convolution2D)  (None, 16, 20, 20)    6416        activation_11[0][0]              
____________________________________________________________________________________________________
activation_12 (Activation)       (None, 16, 20, 20)    0           convolution2d_8[0][0]            
___________________________________________________________________________________________